In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import cross_val_score
from itertools import combinations
import itertools
from sklearn.model_selection import KFold

In [ ]:
df = pd.read_csv('wine.csv', sep=';')
df

In [ ]:
def preprocess(x, y):
    A = np.hstack((np.ones((x.shape[0], 1)), x))
    
    return A, y
def ols_linear_regression(A, b):
    A_pinv = np.linalg.inv(A.T @ A) @ A.T    # np.linalg.pinv(A)
    x = A_pinv @ b
    
    return x
def plot_regression(x, y):
    A, b = preprocess(x, y)
    
    weight = ols_linear_regression(A, b)
    
    # Plot data
    plt.scatter(x, y, color='b', marker='o')
  
    # Plot regression line
    y_pred = weight[0] + weight[1] * x
    plt.plot(x, y_pred, color='r', label='y = {:.2f} + {:.2f}*x'.format(weight[0][0], weight[1][0]))
  
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(loc='lower right')
    
    plt.show()

In [ ]:
#Câu 1a:
x = df.iloc[:, 0:11].values
y = df.iloc[:, 11].values
A, b = preprocess(x, y)
ols_linear_regression(A, b)           #kết quả

In [ ]:
print(np.linalg.norm(A@ols_linear_regression(A, b) - b))       #độ lỗi

In [ ]:
#Câu 1b
B = []
scores = []
for i in range(11):
    X = df.iloc[:, [i]].values
    clf = svm.SVC(kernel='linear', C=1, random_state=13)
    scores = cross_val_score(clf, X, y, cv=5)
    print(scores)
    B = B + [np.mean(scores)]        #mean 

In [ ]:
B

In [ ]:
def tim_min(A):
    t = np.min(A)
    b = c = 0
    for i in range(11):
        b = A[i]
        if (b == t):
            c = i
    return c

temp = tim_min(B)

In [ ]:
X_b = df.iloc[:, temp].values.reshape(-1, 1)

In [ ]:
B_b, b_b = preprocess(X_b, y)

In [ ]:
ols_linear_regression(B_b, b_b)

In [ ]:
print(np.linalg.norm(B_b@ols_linear_regression(B_b, b_b) - b_b))

In [ ]:
plot_regression(X_b, y.reshape(-1, 1))

In [ ]:
#Câu c
def train_test(train, test, X, y):         #hàm tách x dùng để train câu c, dưới là phần của câu c
    X_train = X[train]                        
    y_train = y[train]
    X_test = X[test]
    y_test = y[test]
    return X_train, X_test, y_train, y_test
def MSE(X_train, y_train, X_test, y_test):    
    Regressor = LinearRegression()              #bình phương nhỏ nhất trong hồi qui tuyến tính
    Regressor.fit(X_train, y_train)
    y_pred = Regressor.predict(X_test)
    return metrics.mean_squared_error(y_pred, y_test)    #MSE

C = [i for i in range(11)]        #tạo mảng comb chưa tất cả các trường hợp tổ hợp các cột 
Comb = []
for i in range(1, 12):
    comb = list(combinations(C, i))
    Comb += [comb]
    
newComb = []                     #tổ hợp các trường hợp không lặp 
for x in Comb:                   
    for Xith in x:
        newComb += [np.array(Xith)]


In [ ]:
score = []
kfold = KFold(n_splits = 5, random_state=13, shuffle=True)

for attribute in newComb:
    scoreI = []
    X = df.iloc[:, attribute].values
    for train, test in kfold.split(X):
        X_train, X_test, y_train, y_test = train_test(train, test, X, y)
        scoreI = scoreI + [MSE(X_train, y_train, X_test, y_test)]
    score = score + [np.mean(np.array(scoreI))]
    temp = temp + [scoreI]

In [ ]:
pos = score.index(min(score))
pos

In [ ]:
newComb[pos]

In [ ]:
X_c = df.iloc[:, newComb[pos]].values
A, b = preprocess(X_c, y)
ols_linear_regression(A, b)

In [ ]:
print(np.linalg.norm(A@ols_linear_regression(A, b) - b))

In [ ]:
(scoreI)

In [ ]:
np.mean(temp[5])